<h2>Importing Data and Librarires</h2>

In [17]:
import numpy as np
import pandas as pd
from future.utils import iteritems
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import sys

if not sys.warnoptions: # ignore warnings 3ahsan e7na rewsheen gedan
    import warnings
    warnings.simplefilter("ignore")

In [18]:
gtd17 = pd.read_csv('globalterrorismdb_0718dist.csv',encoding='iso-8859-1',low_memory=False)
C = pd.read_csv('history.csv',encoding='iso-8859-1',low_memory=False)


In [19]:
C.columns

Index(['CPI 2016 Rank', 'Country', 'Country Code', 'Region', 'CPI 2016 Score',
       'CPI 2015 Score', 'CPI 2014 Score', 'CPI 2013 Score', 'CPI 2012 Score'],
      dtype='object')

In [20]:
C1 = C[['Country','CPI 2012 Score']]
C1=C1.rename(columns={'CPI 2012 Score':"score"})
C1['year'] = 2012
C2 = C[['Country','CPI 2013 Score']]
C2['year'] = 2013
C2=C2.rename(columns={'CPI 2013 Score':"score"})
C3 = C[['Country','CPI 2014 Score']]
C3['year'] = 2014
C3=C3.rename(columns={'CPI 2014 Score':"score"})
C4 = C[['Country','CPI 2015 Score']]
C4['year'] = 2015
C4 = C4.rename(columns={'CPI 2015 Score':"score"})
C5= C[['Country','CPI 2016 Score']]
C5['year'] = 2016
C5 = C5.rename(columns={'CPI 2016 Score':"score"})
Cor = pd.concat([C1,C2,C3,C4,C5,])

In [21]:
df = gtd17
df2 = Cor
names = {}
for name in df2.columns:
    names[name] = str(name)+'_c'

df2 = df2.rename(columns=names)
df.iyear.astype(int)
df2.year_c.astype(int)
df.country_txt.astype(str)
df2.Country_c.astype(str)
join = pd.merge(df,df2, left_on=['country_txt','iyear'] ,right_on=['Country_c','year_c'] ,how='outer')

<h2>Preprocessing GTD + CPI</h2>

In [22]:
# MENA regions only
ds = join[join['region_txt'] == 'Middle East & North Africa']
# Year 2000-2017
ds = ds[ds['iyear'] > 1999]
# removing incidents where group is unknown
ds = ds[ds['gname'] != 'Unknown']
# choosing top groups
votes = {}
for name in ds['gname']:
    votes[name] = votes.get(name,0)+1

counts = []
names = []
for name,count in iteritems(votes):
    names.append(name)
    counts.append(count)
    
names = [name for _,name in sorted(zip(counts,names))]
names = names[-5:]
contained = [name in names for name in ds['gname']]
ds = ds[contained]
# encoding gname
lb_make = LabelEncoder()
ds["gname"] = lb_make.fit_transform(ds["gname"])
# drop redundant col , for ex region and region_txt , drop region
names = list(set(ds))
drop = ['region','region_txt','eventid','gname']
for col in names:
    f = str(col)+'_txt'
    if (f) in names and f not in drop :
            drop.append(f) # for ex if 


<h2>Split train: incidents 2000-2016 , test: incidents 2017</h2>

In [23]:
Xtrain = ds[ds['iyear']<2017]
Ytrain = Xtrain['gname']
Xtrain = Xtrain.drop(columns=drop)
Xtest = ds[ds['iyear']==2017]
Ytest = Xtest['gname']
Xtest = Xtest.drop(columns=drop)


<h2>Getting best features</h2>

In [24]:
votes = {}
for name in Ytrain:
    votes[name] = votes.get(name,0)+1

print('number of attakcs per each group in train ' ,votes)
votes = {}
for name in Ytest:
    votes[name] = votes.get(name,0)+1

print('number of attakcs per each group in test ' ,votes)

number of attakcs per each group in train  {4: 1068, 0: 637, 1: 973, 2: 904, 3: 4271}
number of attakcs per each group in test  {3: 1292, 4: 155, 2: 158, 1: 44}


In [25]:
def get_features(model , Xtrain , Ytrain):
    d = Xtrain.shape[1]
    num = len(Xtrain.columns) # total number of features
    best_scores = []
    names = list(Xtrain.columns) # names of all features
    Y = Ytrain 
    best_scores = []
    good_features = [] # stores the names of the best selected features
    bad_features = [] # stores the names of the  bad features not to try with again
    for i in range(5) : # take best 5 features
        scores = []
        
        for j,col in enumerate(Xtrain): # search for the next best feature that is the one which increases accuracy the most
            percent = round((i*d+j)/(d*5)*100,0)
            print('Loading '+str(percent)+' %',end="\r")
           
            if col in good_features or col in bad_features: # if feature reviewed before as good or bad skip
                scores.append(0)
                continue
            feature = Xtrain[col]  # get each feature
            X = Xtrain[good_features]  # get previous selected features
            #X.loc[:,col] = feature   # add this feature
            X = X.assign(col=feature)
            if 'O' in list(set(X.dtypes)): # one of the features is not numeric and needs encoding , note that nan values become zeros
                X = pd.get_dummies(X)
            X = X.fillna(X.mean()) # replace nan values by mean in numeric features
            if len (X.columns) > 20: # one-hot encoding results in too many dimensions , skip
                scores.append(0)
                continue
            if len (X.columns) == 0: # dont know why this is here
                scores.append(0)
                continue
            if X.isnull().values.any(): # a feauture still has nan values , this occurs if it initially has only nan values , it may had values in the removed columns
                scores.append(0)
                continue
           
            model.fit(X,Y)

            score = model.score(X,Y)
            if score < 0.02: # features that score less than 2% are bad and should not be reconsidered
                scores.append(0)
                bad_features.append(col)
            else:
                scores.append(score)
                
        index = scores.index(max(scores)) # get best feature
        best_scores.append(max(scores)) # store best score for 1 to n features
        good_features.append(names[index])
    return (good_features,best_scores)

In [26]:
def get_score(model,good_features,ds): # return the score after selecting best features
    scores = []
    fscores = []
    for i in range(5):
        features=good_features.copy()
        features = features[:i+1] 
        selected = features.copy()
        if 'iyear' not in features:
            selected.append('iyear')
        selected.append('gname')
        df = ds[selected]
        df = pd.get_dummies(df)
        df = df.fillna(df.mean())

        Xtrain = df[df['iyear']<2017]
        Ytrain = Xtrain['gname']
        Xtrain = Xtrain.drop(columns=['iyear','gname']) # always drop iyear , since test and trai wikk akways have different years
        Xtest = df[df['iyear']==2017]
        Ytest = Xtest['gname']
        Xtest = Xtest.drop(columns=['iyear','gname'])
        model.fit(Xtrain,Ytrain)
        scores.append(model.score(Xtest,Ytest))
        fscores.append(f1_score(Ytest,model.predict(Xtest),average='weighted'))
    return (max(scores),max(fscores))

<h2>KNN</h2>

In [27]:
model = KNeighborsClassifier(n_neighbors=2)
good_features,best_scores = get_features(model,Xtrain,Ytrain)
print('good features ',good_features)
print('train scores :',best_scores)
score,fscore = get_score(model,good_features,ds)
print('test score ',score)
print('test fscore ',fscore)

good features  ['latitude', 'INT_IDEO', 'score_c', 'INT_MISC', 'iyear']
train scores : [0.7911626130141347, 0.9773335031198268, 0.9932509868839934, 0.9947790653253533, 0.9963071437667134]
test score  0.9666464523953912
test fscore  0.9720007588603293


<h2>Decision Trees</h2>

In [28]:
model = DecisionTreeClassifier()
good_features,best_scores = get_features(model,Xtrain,Ytrain)
print('good features ',good_features)
print('train scores :',best_scores)
score,fscore = get_score(model,good_features,ds)
print('test score ',score)
print('test fscore ',fscore)

good features  ['latitude', 'INT_IDEO', 'score_c', 'targsubtype1', 'iday']
train scores : [0.8976187444288807, 0.9893034509104801, 0.9966891633770534, 0.9989812810390933, 1.0]
test score  0.9733171619163129
test fscore  0.9751186974252035


<h2>Naive Bayes</h2>

In [29]:
model = GaussianNB() 
good_features,best_scores = get_features(model,Xtrain,Ytrain)
print('good features ',good_features)
print('train scores :',best_scores)
score,fscore = get_score(model,good_features,ds)
print('test score ',score)
print('test fscore ',fscore)

good features  ['latitude', 'INT_IDEO', 'country', 'dbsource', 'nperpcap']
train scores : [0.801222462753088, 0.9598879409143003, 0.9834458168852668, 0.9882847319495734, 0.9885394116898001]
test score  0.976349302607641
test fscore  0.9778590147020995


<h2>SVM</h2>

In [30]:
model = LinearSVC() 
good_features,best_scores = get_features(model,Xtrain,Ytrain)
print(good_features)
print(best_scores)
score,fscore = get_score(model,good_features,ds)
print('test score ',score)
print('test fscore ',fscore)

['Country_c', 'INT_IDEO', 'country', 'INT_MISC', 'latitude']
[0.8036419202852413, 0.9661275945498535, 0.9834458168852668, 0.9868839933783268, 0.9899401502610468]
test score  0.9502728926622195
test fscore  0.9560876366268388


<h2>Random Forest</h2>

In [31]:
model = RandomForestClassifier() 
good_features,best_scores = get_features(model,Xtrain,Ytrain)
print(good_features)
print(best_scores)
score,fscore = get_score(model,good_features,ds)
print('test score ',score)
print('test fscore ',fscore)

['latitude', 'INT_IDEO', 'score_c', 'natlty1', 'targtype1']
[0.8940532280657074, 0.9884120718196867, 0.9959251241563734, 0.9983445816885267, 0.9991086209092067]
test score  0.9727107337780473
test fscore  0.9745140841777493


<h2>Quadratic Discriminant Analysis</h2>

In [32]:
model = QuadraticDiscriminantAnalysis() 
good_features,best_scores = get_features(model,Xtrain,Ytrain)
print(good_features)
print(best_scores)
score,fscore = get_score(model,good_features,ds)
print('test score ',score)
print('test fscore ',fscore)

['latitude', 'iyear', 'longitude', 'natlty1', 'claimed']
[0.801222462753088, 0.8822106201451675, 0.902330319623074, 0.9297083916974405, 0.9374761237743537]
test score  0.9617950272892662
test fscore  0.9624363294546656
